In [1]:
%run pretrained-model.ipynb

CUDA Available:  True
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)


In [2]:
import random
import torchvision
torch.manual_seed(17)

In [3]:
# FGSM attack code
def fgsm_attack(image, alpha, data_grad):
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + alpha*data_grad.sign()
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [4]:
def diversify(image, prob=0.5, rand_range=(22,27)):  
    """Input diversify method

    image:      The input image X
    prob:       The transform probability
    rand_range: The range of random size to resize to
    
    return:     The transformed image 
    """ 
    if random.uniform(0, 1) < prob:
        return image
        
    # random resize
    rand_from, rand_to = rand_range
    rnd = random.randint(rand_from, rand_to)
    resize = torchvision.transforms.Resize((rnd, rnd))
    
    # random padding
    remained = 28 - rnd
    pad_top = random.randint(0, remained)
    pad_bottom = remained - pad_top
    pad_left = random.randint(0, remained)
    pad_right = remained - pad_left
    pad = torchvision.transforms.Pad(padding=[pad_top, pad_left, pad_bottom, pad_right], 
                                     fill=0, 
                                     padding_mode='constant')
    
    # apply transformation
    resized = resize(image)
    padded  = pad(resized)

    return padded
    

In [14]:
def mdiifgsm(image, label, epsilon, iters, decay, prob, rand_range):
    """Perform I-FGSM attack on an image

    image:     The input image X
    label:     The image label y
    epsilon:   The adversarial perturbation size
    iters:     The number of iterations
    
    return:    The adversatial image X*
    """ 
    grad = 0 
    for _ in range(iters):
        # Setting this is important for the attack
        image.requires_grad = True

        # Obtain the gradients
        output = model(diversify(image, 
                                 prob=prob,
                                 rand_range=rand_range).to(device))
        loss = F.nll_loss(output, label)
        model.zero_grad()
        loss.backward()

        # gather gradients for the first n iterations with a small decay
        grad = (decay * grad) + (image.grad.data / torch.norm(image.grad.data, p=1))
        
        # Create the perturbed image by adjusting each pixel of the input image
        image = image + epsilon * grad.sign()
        image = torch.clamp(image, min=0, max=1).detach()
    
    return image

In [15]:
correct = 0
test_loader_iter = iter(test_loader)

for j in range(1000):
    image, label = next(test_loader_iter)
    
    initial_prediction = torch.argmax(model(image.to(device))).item()
    # Don't bother attacking if the image is already misclassified
    if initial_prediction != label:
        continue

    # Generate FGSM adversarial example
    adv_image = mdiifgsm(image      = image.to(device), 
                         label      = label.to(device), 
                         epsilon    = 0.1, 
                         iters      = 3, 
                         decay      = 1.0, 
                         prob       = 0.5, 
                         rand_range = (22,27))

    final_prediction = torch.argmax(model(adv_image)).item()

    # Correct if the prediction is the target label
    if final_prediction != label:
        correct += 1
    
# Calculate final accuracy for this epsilon    
accuracy = correct/1000

In [16]:
accuracy

0.964